# Kuramoto: Curriculum Training
Below you may find the corresponding trainning proceedure for Kuramoto.
This proceedure has been adapted from a script and is expected to fit and produce a NODEC that can control oscillator graphs.

Please make sure that the required data folder is available at the paths used by the script.
You may generate the required data by running the python script
```nodec_experiments/kuramoto/gen_parameters.py```.

As neural network intialization is stochastic, please make sure that appropriate seeds are used or expect some variance to paper results.
This can be evident as sometimes training does not yield a stable controller.


## Imports

In [ ]:
# when developing
# %load_ext autoreload
# %autoreload 2

In [2]:
#将仓库根目录（nnc-master/）添加到 Python 系统路径中。
#为什么需要？：脚本位于 nnc-master/nodec_experiments/kuramoto/，而要导入的 nnc/ 模块在根目录下。通过 sys.path.append('../../')，Python 才能找到 nnc.controllers、nnc.helpers 等自定义模块，避免导入错误。
import os
os.sys.path.append('../../')

In [3]:
#基础工具库导入
import math
import torch
from torchdiffeq import odeint
import numpy as np
import pandas as pd
import networkx as nx

#可视化工具库导入
import plotly.express as px
from plotly import graph_objects as go

#Kuramoto 控制相关的模块导入
from nnc.controllers.neural_network.nnc_controllers import NNCDynamics
from nnc.controllers.baselines.oscillators.dynamics import AdditiveControlKuramotoDynamics
from nnc.controllers.baselines.oscillators.optimal_controllers import KuramotoFeedbackControl

from nnc.helpers.torch_utils.graphs import adjacency_tensor, maximum_matching_drivers, drivers_to_tensor
from nnc.helpers.torch_utils.oscillators import order_parameter_cos
#优化版 ODE 求解器，优化计算速度，避免重复计算邻接矩阵相关的操作，尤其适合大规模振子系统
from nnc.helpers.torch_utils.numerics import faster_adj_odeint
from nnc.helpers.plot_helper import ColorRegistry, base_layout


from tqdm.cli import tqdm

C:\Users\hp\.conda\envs\ANNF\lib\site-packages\pandas\_testing.py:24: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  import pandas._libs.testing as _testing


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# Script Params

# Torch params
device = 'cuda:0'
dtype = torch.float 

# NODEC Params
train = True         # retrain or load pretrained model


# Feedback Control Params


# Paths
data_folder = '../../../data/parameters/kuramoto/'
result_folder = '../../../results/kuramoto/'
os.makedirs(result_folder, exist_ok = True)

graph = 'complete'
graph_folder = data_folder + graph + '/'

## Loading graph and dynamics parameters

In [ ]:
# Loading Parameters for the graph


A = torch.load(graph_folder + 'adjacency.pt', map_location=device).float() # adjacency matrix
G = nx.from_numpy_matrix(A.numpy())
n_nodes = G.number_of_nodes()
mean_degree = np.mean(list(dict(G.degree()).values()))

A = A.to(device, dtype) # adjacency
L = A.sum(-1).diag() - A # laplacian

# to save results
os.makedirs(graph_folder, exist_ok=True)

In [ ]:
# Load dynamics dependendent variables and states
coupling_constants = torch.load(data_folder + 'coupling_constants.pt', map_location=device).to(device, dtype)
frustration_constants = torch.load(data_folder + 'frustration_constants.pt', map_location=device).to(device, dtype)
natural_frequencies = torch.load(data_folder + 'nominal_angular_velocities.pt', map_location=device).to(device, dtype)
K = coupling_constants[2].item() # coupling constant, index 2 should be 0.4
frustration_constant = frustration_constants[0] # we use no frustration for this example
dynamics_params_folder = graph_folder + 'dynamics_parameters/coupling_' + '{:.1f}'.format(K) + '/'


x0 = torch.load(data_folder + 'single_init.pt',  map_location=device)


# to avoid using extra memory we load the driver vector 
# and use element-wise multiplication instead of the driver matrix.
gain_vector = torch.load(dynamics_params_folder + 'driver_vector.pt', map_location=device).to(device, dtype)
driver_nodes = torch.nonzero(gain_vector).cpu().numpy().flatten().tolist()
driver_percentage = len(driver_nodes)/len(gain_vector)
steady_state = torch.load(dynamics_params_folder + 'steady_state.pt', map_location=device).to(device, dtype)


## Preparing Feedback Control Baseline Parameters

In [ ]:
#  Controller parameters
# Feedback Control
feedback_control_constant = 10

# Neural Network training
n_hidden_units = 3
batch_size = 8 # for code ocean GPUs this might be too much, 
               # reduce to 4 or 2 but stability of learned control may suffer.
epochs = 20

In [ ]:
print('Current experiment info:')
print('\t Loaded ' + graph + 'graph with: ' + str(n_nodes) + ' nodes and ' + str(G.number_of_edges()) + ' edges.' )
print('\t Coupling Constant: ' + str(K))
print('\t Frustration Constant: ' + str(frustration_constant.item()))
print('\t Natural Frequencies: mean: ' + str(natural_frequencies.mean().item()) + ' variance: ' + str(natural_frequencies.var().item()) )
print('\t Ratio of driver node vs total nodes: '  + str(len(driver_nodes)/n_nodes))
print('\t Feedback Control Constant: '  + str(feedback_control_constant))


In [ ]:
# Generating the dynamics:
dyn = AdditiveControlKuramotoDynamics(
    A, 
    K, 
    natural_frequencies,
    frustration_constant=frustration_constant
).to(device)

### Results without control

In [ ]:
# Generating a trajectory without control
tlin = torch.linspace(0, 150, 500).to(device)
state_trajectory_noc = odeint(lambda t,y: dyn(t,y,u=None),x0, tlin, method='dopri5')
y=order_parameter_cos(state_trajectory_noc.squeeze().cpu())
fig_noc = px.line(y=y.cpu().numpy(), x=tlin.cpu().numpy())
fig_noc.data[0].name = 'No control'
fig_noc.data[0].line.color = ColorRegistry.constant
fig_noc.data[0].showlegend = True
fig_noc.layout.xaxis.title.text = 'Time'
fig_noc.layout.yaxis.title.text = '$r(t)$'
fig_noc

### Results with feedback control

In [ ]:
# Generating feecback control trajectory
cont = lambda x: feedback_control_constant*gain_vector*torch.sin(-x)
state_trajectory_oc = odeint(lambda t,y: dyn(t,y.detach(),u=cont(y).detach()), 
            x0,
            torch.linspace(0, 150, 500).to(device), 
            method='dopri5'
           )
y=order_parameter_cos(state_trajectory_oc.squeeze().cpu())
fig_fc = px.line(y=y.cpu().numpy(), x=torch.linspace(0, 150, 500).numpy())
fig_fc = px.line(y=y.cpu().numpy(), x=tlin.cpu().numpy())
fig_fc.data[0].name = 'Feedback Control'
fig_fc.data[0].line.color = ColorRegistry.oc
fig_fc.data[0].showlegend = True
fig_fc.layout.xaxis.title.text = 'Time'
fig_fc.layout.yaxis.title.text = '$r(t)$'
fig_fc

## NODEC
Since we developed Kuramoto recently, we choose to provide the curriculum learning and the architecture in the notebook.
After unit testing we will include it in the main library.

In [ ]:
# Feedback control neural network
class EluFeedbackControl(torch.nn.Module):
    """
    Very simple Elu architecture for control of linear systems
    """
    def __init__(self, n_nodes, n_drivers, driver_matrix, n_hidden=3):
        super().__init__()
        self.linear = torch.nn.Linear(n_nodes,n_hidden)
        self.linear_h1 = torch.nn.Linear(n_hidden, n_hidden)
        self.linear_final = torch.nn.Linear(n_hidden, n_drivers)
        self.driver_matrix = driver_matrix

    def forward(self, t, x):
        """
        :param t: A scalar or a batch with scalars
        :param x: input_states for all nodes
        :return:
        """     
        u = self.linear(torch.sin(x))
        u = torch.nn.functional.elu(u)
        u = self.linear_h1(u)
        u = torch.nn.functional.elu(u)
        u = self.linear_final(u)
        # we multiply by the nn driver matrix to generate the control signal
        u = (self.driver_matrix@u.unsqueeze(-1)).squeeze(-1)
        return u

In [ ]:
# We convert the driver vector back to a matrix 
# and convert the non-zero elements to 1, so that the neural network is agnostic of the exact gain values.
driver_matrix = drivers_to_tensor(A.shape[-1], driver_nodes).to(dtype=dtype, device=device)

In [ ]:
# we set a seed for NN weight generation to try to make results as reproducible as possible:
torch.manual_seed(3548)
neural_net = EluFeedbackControl(n_nodes,
                                len(driver_nodes), 
                                driver_matrix,
                                n_hidden=n_hidden_units
                               ).to(dtype=dtype, device=device)

for param, dat in neural_net.named_parameters():
    if 'bias' not in param:
        torch.nn.init.xavier_normal_(dat)
        # we initialize close to 0 to avoid learning high energy solutions
        dat = dat/1000
nnc_dyn = NNCDynamics(dyn, neural_net).to(dtype=dtype, device=device)


In [ ]:
optimizer = torch.optim.Adam(neural_net.parameters(), lr=0.1)

###  Training with curriculum
Please notice how the batch trainning and curriculum learning proceedures operate on lines 8 and 24 respectively.

In [ ]:
if train:
    pbar = tqdm(range(epochs))
    trajectory_length = [1]
    for i in pbar:
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        def closure():
            # sample new minibatch for training
            sample = torch.randn([batch_size, 1, n_nodes]).to(device)
            state_samples = 100
            x_reached = faster_adj_odeint(nnc_dyn, 
                                          sample,
                                          torch.linspace(0,
                                                         trajectory_length[0],
                                                         state_samples).to(device), 
                                       method='dopri5', 
                               #adjoint_params=neural_net.parameters()
                              )[1:]        

            op = order_parameter_cos(x_reached)
            loss =  (-op.mean(-1) - op.min(-1).values).mean()       
            loss.backward()
            loss_value = loss.item()
            pbar.set_postfix({'Training loss: ' :  str(round(loss_value,2)) ,  
                              'trajectory length in time units: ' : str(round(trajectory_length[0], 2))})
            # increase trajectory by sampling a uniform distribution in [0,2]
            trajectory_length[0] = trajectory_length[0]+2*torch.rand(1).item()
            return loss.item()
        

        try:
            optimizer.step(closure)
        except Exception as e:
            raise e
    torch.save(neural_net.state_dict(), result_folder + 'trained_model.pt')
else: 
    neural_net.load_state_dict(torch.load( '../../../data/parameters/kuramoto/erdos_renyi/trained_model.pt',  
                                          map_location=device)
                              )

In [ ]:
torch.save(neural_net.state_dict(), result_folder + 'trained_model.pt')

In [ ]:
optimizer.zero_grad()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
tlin = torch.linspace(0, 150, 500).to(device)
state_trajectory_nn = odeint( lambda t,y: dyn(t,y.detach(),u=nnc_dyn.nnc.neural_net(t, y.detach())), 
            x0,
            tlin, method='dopri5',
           )


### NODEC Results

In [ ]:
y = order_parameter_cos(state_trajectory_nn.cpu().detach())

px.line(y=y.cpu().numpy().flatten(), x = tlin.cpu().numpy())